This notebook will be about applying some simple methods that are not too complicated.

We want to treat our imaginary end-users as business stakeholders who have a problem and need a short-term solution (day 1 answer)

Given this context, lets come up with some easy answers.

In [48]:
import pandas as pd

### Using current leaderboard ranking, recent form and historical rivalries

In [49]:
# Things to add:
# Goals scored and conceded by both home and away team
# Form of home and away team
# Betting odds

In [50]:
def adjusted_probs(home_odds, draw_odds, away_odds):

    total_probs = sum([(1/odds) * 100 for odds in [home_odds, draw_odds, away_odds]])

    return tuple(((1/odds) * 100 )/total_probs for odds in [home_odds, draw_odds, away_odds])

In [51]:
matches_df = (
    pd.read_csv('../data/N1.csv')
    .filter(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'B365H', 'B365D', 'B365A'])
    .rename(columns=str.lower)
    .rename(columns = {
        'b365h': 'odds_home', 
        'b365d': 'odds_draw', 
        'b365a': 'odds_away'
    })
    .assign(
        date=lambda x: pd.to_datetime(x['date'], format='%d/%m/%Y'),
        adj_home=lambda x: x.apply(lambda row: adjusted_probs(row['odds_home'], row['odds_draw'], row['odds_away'])[0], axis=1),
        adj_draw=lambda x: x.apply(lambda row: adjusted_probs(row['odds_home'], row['odds_draw'], row['odds_away'])[1], axis=1),
        adj_away=lambda x: x.apply(lambda row: adjusted_probs(row['odds_home'], row['odds_draw'], row['odds_away'])[2], axis=1),
    )
)

In [52]:
matches_df

,date,hometeam,awayteam,fthg,ftag,ftr,odds_home,odds_draw,odds_away,adj_home,adj_draw,adj_away
0,2023-08-11,Volendam,Vitesse,1,2,A,3.40,3.75,2.0,0.277264,0.251386,0.471349
1,2023-08-12,PSV Eindhoven,Utrecht,2,0,H,1.36,5.00,8.5,0.698324,0.189944,0.111732
2,2023-08-12,Heerenveen,Waalwijk,3,1,H,1.73,4.00,4.5,0.550375,0.238037,0.211588
3,2023-08-12,Ajax,Heracles,4,1,H,1.20,7.00,13.0,0.791304,0.135652,0.073043
4,2023-08-12,Zwolle,Sparta Rotterdam,1,2,A,2.60,3.40,2.7,0.366613,0.280351,0.353035
...,...,...,...,...,...,...,...,...,...,...,...,...
130,2023-12-09,Zwolle,Nijmegen,1,3,A,2.38,3.50,2.9,0.399890,0.271925,0.328185
131,2023-12-09,AZ Alkmaar,Almere City,4,1,H,1.18,7.00,15.0,0.801772,0.135156,0.063073
132,2023-12-10,Go Ahead Eagles,Utrecht,0,2,A,2.15,3.40,3.4,0.441558,0.279221,0.279221
133,2023-12-10,For Sittard,Waalwijk,1,0,H,1.91,3.60,4.0,0.497994,0.264214,0.237792


In [53]:
league_tables = []

In [54]:
gameweek_1 = matches_df.query('date < "2023-08-12"')
gameweek_1

,date,hometeam,awayteam,fthg,ftag,ftr,odds_home,odds_draw,odds_away,adj_home,adj_draw,adj_away
0,2023-08-11,Volendam,Vitesse,1,2,A,3.4,3.75,2.0,0.277264,0.251386,0.471349


In [55]:
league_table = {
    team: {
        'points': 0,
        'goals_for': 0,
        'goals_against': 0,
        'form': []
    } for team in matches_df.hometeam.unique()
}

In [56]:
date = '2023-08-11'

league_tables.append(league_table)

In [57]:
pd.DataFrame.from_dict(league_tables[pd.Timestamp('2023-08-11')], orient = 'index')

,points,goals_for,goals_against,form
Volendam,0,0,0,[]
PSV Eindhoven,0,0,0,[]
Heerenveen,0,0,0,[]
Ajax,0,0,0,[]
Zwolle,0,0,0,[]
Nijmegen,0,0,0,[]
AZ Alkmaar,0,0,0,[]
Feyenoord,0,0,0,[]
Almere City,0,0,0,[]
Heracles,0,0,0,[]


In [58]:
list_of_games = gameweek_1.to_dict(orient='records')

game = list_of_games[0]

league_table[game['hometeam']]['goals_for'] += game['fthg']
league_table[game['hometeam']]['goals_against'] += game['ftag']

league_table[game['awayteam']]['goals_for'] += game['ftag']
league_table[game['awayteam']]['goals_against'] += game['fthg']

if game['ftr'] == 'H':
    league_table[game['hometeam']]['points'] += 3
    league_table[game['hometeam']]['form'].append('W')
    league_table[game['awayteam']]['form'].append('L')

if game['ftr'] == 'A':
    league_table[game['awayteam']]['points'] += 3
    league_table[game['awayteam']]['form'].append('W')
    league_table[game['hometeam']]['form'].append('L')

In [59]:
pd.DataFrame.from_dict(league_table, orient = 'index')

,points,goals_for,goals_against,form
Volendam,0,1,2,[L]
PSV Eindhoven,0,0,0,[]
Heerenveen,0,0,0,[]
Ajax,0,0,0,[]
Zwolle,0,0,0,[]
Nijmegen,0,0,0,[]
AZ Alkmaar,0,0,0,[]
Feyenoord,0,0,0,[]
Almere City,0,0,0,[]
Heracles,0,0,0,[]
